In [1]:
# install library w rec engines in it
!pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install numpy==1.24.4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

from surprise import Dataset, Reader

from surprise import KNNBasic

from surprise.model_selection import cross_validate

In [4]:
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')

movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')


In [5]:
#merge data together

data = pd.merge(ratings_df, movies_df, on='movieId')

In [6]:
reader = Reader(rating_scale=(1,10))

data = Dataset.load_from_df(data[['userId','movieId','rating']], reader)

In [7]:
sim_options = {'name':'cosine',

               'user_based': False} # content based filtering

In [8]:
algo = KNNBasic(sim_options=sim_options)


In [9]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9785  0.9752  0.9760  0.9739  0.9698  0.9747  0.0028  
Fit time          2.63    2.72    2.65    2.33    2.30    2.53    0.18    
Test time         2.62    2.77    2.49    2.63    2.47    2.59    0.11    


{'test_rmse': array([0.97846333, 0.97516714, 0.97597077, 0.97391205, 0.96982157]),
 'fit_time': (2.6335408687591553,
  2.7203869819641113,
  2.651120901107788,
  2.3313019275665283,
  2.301574945449829),
 'test_time': (2.616281032562256,
  2.7686619758605957,
  2.487809896469116,
  2.6293938159942627,
  2.472350835800171)}

In [10]:
# fit data to algorithm

algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [11]:
#print similarities between movies, gives matrix

print(algo.sim)

[[1.         0.90773108 0.96657344 ... 1.         1.         1.        ]
 [0.90773108 1.         0.88581521 ... 1.         1.         1.        ]
 [0.96657344 0.88581521 1.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]


In [12]:
#can use to make predictions for specific users
pred = algo.predict(uid=1, iid=10)

pred.est

3.9880309408033616

In [13]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [14]:
#this is how item based algorithm will work
sim_options = {'name':'cosine',

               'user_based': False,

               'k':10} # content based filtering


In [15]:
algo = KNNBasic(sim_options=sim_options)


In [16]:
#results are pretty much the same but this is hwo u can adjust the parametre
#can visualize heatmap using seaborn but wont run rn bcz will take very long
sns.heatmap(np.array(similarities), annot=True)

NameError: name 'sns' is not defined

In [ ]:
# will tell you which are similar

User based filtering

In [17]:
from surprise.model_selection import train_test_split

from surprise import accuracy


In [18]:
trainset, testset = train_test_split(data, test_size=0.25)


In [19]:
# Using the same initial setup as in the user-based example



# Using KNN to compute similarities between items

algo = KNNBasic(sim_options={'user_based': False})

algo.fit(trainset)



# Making predictions and evaluating the model

predictions = algo.test(testset)

accuracy.rmse(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9137


0.9137330477021934

In [20]:
algo = KNNBasic(sim_options={'user_based':True})


In [21]:
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [22]:
predictions = algo.test(testset)

In [23]:
predictions

[Prediction(uid=666, iid=1258, r_ui=5.0, est=4.085326570963338, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=575, iid=3385, r_ui=3.0, est=3.516125765784011, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=22, iid=3978, r_ui=4.0, est=3.0954621115400878, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=24, iid=3364, r_ui=4.0, est=4.37580995925466, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=558, iid=281, r_ui=3.0, est=3.6604369416621694, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=224, iid=8814, r_ui=2.0, est=3.1331493689802654, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=413, iid=40583, r_ui=2.5, est=3.8179752226557837, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=544, iid=2012, r_ui=4.0, est=3.7736904954797805, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=668, iid=7451, r_ui=2.5, est=3.6

In [24]:
df[(df['userId']==668) & (df['movieId']==65088)]


,userId,movieId,rating,timestamp,title,genres
103973,668,65088,2.0,1261893683,Bedtime Stories (2008),Adventure|Children|Comedy


In [25]:
#can see how far off we are
#from model can see pretty accurate for first few entries, so can be used to make predicitons where dont have rating

#predict specfic user
df[df['userId']==1]['movieId'].unique()


array([   16,    24,    32,    47,    50,   110,   150,   161,   165,
         204,   223,   256,   260,   261,   277,   296,   318,   349,
         356,   377,   380,   457,   480,   527,   589,   590,   592,
         593,   597,   608,   648,   719,   724,   736,   780,   858,
         912,   968,  1061,  1089,  1136,  1196,  1198,  1210,  1213,
        1220,  1221,  1222,  1233,  1243,  1258,  1265,  1267,  1270,
        1287,  1580,  1617,  1721,  1923,  1961,  2021,  2028,  2105,
        2161,  2194,  2396,  2407,  2455,  2467,  2502,  2542,  2571,
        2628,  2716,  2728,  2762,  2858,  2947,  2959,  3256,  3421,
        3578,  4011,  4027,  4033,  4085,  4262,  4306,  4963,  4993,
        4995,  5349,  5378,  5418,  5445,  5952,  6365,  6711,  6807,
        7153,  8825,  8961, 32587, 33493, 33794, 33834, 45950, 48516,
       48780, 49272, 52973, 54286, 57949])

In [ ]:
# want to see prediction for movie 10, can see high rating so would reccomend
#previously with item based was 3.98 but when looked at similar users see 3.58
# checking what movie it is: golden eye
#can choose which appoach we want to go with for this problem

In [26]:
# SVD



from surprise import SVD

In [27]:
#can test if good model for problem
#root mean square lower for this one so maybe better
svd = SVD()

cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8646  0.8685  0.8713  0.8672  0.8756  0.8695  0.0038  
Fit time          0.35    0.34    0.35    0.34    0.35    0.35    0.00    
Test time         0.08    0.04    0.08    0.04    0.04    0.06    0.02    


{'test_rmse': array([0.86459017, 0.8685452 , 0.87132698, 0.86718034, 0.8756125 ]),
 'fit_time': (0.3507692813873291,
  0.3428962230682373,
  0.34825897216796875,
  0.34319281578063965,
  0.34803104400634766),
 'test_time': (0.08116984367370605,
  0.04057431221008301,
  0.08062124252319336,
  0.042571067810058594,
  0.04198908805847168)}

In [28]:
trainset2 = data.build_full_trainset()

svd.fit(trainset)


In [29]:
svd.predict(1, 10)

Prediction(uid=1, iid=10, r_ui=None, est=3.295566062197506, details={'was_impossible': False})

In [30]:
# predictions for everyone



df['predicted_rating'] = df.apply(lambda x: svd.predict(x['userId'], x['movieId']).est, axis=1)


In [31]:
# can see predicitons vs actual so can see how model is actual performing
# user 12, movie 16 can see its off
df

,userId,movieId,rating,timestamp,title,genres,predicted_rating
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama,3.764587
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi,2.061959
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,3.778558
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,3.824520
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.068249
...,...,...,...,...,...,...,...
105334,668,142488,4.0,1451535844,Spotlight (2015),Thriller,2.756355
105335,668,142507,3.5,1451535889,Pawn Sacrifice (2015),Drama,2.756355
105336,668,143385,4.0,1446388585,Bridge of Spies (2015),Drama|Thriller,3.370393
105337,668,144976,2.5,1448656898,Bone Tomahawk (2015),Horror|Western,3.064855


In [ ]:
#svd is using user and item relationships indirectly
#using all info of both to try to make prediciton
#so doesnt compute direct similarities like how KNN does

#now have 3 ways to make rec engines
#item based, user based, svd: infers missing data points
#can also use trad ML to predict probabilities, and even deep learning
